In [2]:
#DISPLAY JUST IMAGE WITH WORKING DISTORTION BUT SEEMS TO BE TOO BIG
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import ctypes
import math
ctypes.windll.shcore.SetProcessDpiAwareness(1)  # Enable DPI scaling fix

# Initialize Tkinter
root = tk.Tk()
root.title("Capture Card Feed with Transformations")
root.overrideredirect(True) #removes GUI elements/title bar
root.geometry("1920x1080") #size fits the projector screen
root.geometry('+2256+0') #moves window to start of projector screen
root.configure(bg="black")

# Open the Capture Card
cap = cv2.VideoCapture(1)  # Change index if necessary
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 812)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 650) #812x650 is the size of fluoro computer images
cap.set(cv2.CAP_PROP_FPS, 30)

# Create a label to display video
label = tk.Label(root, bg="black") #turned background of window black


# Transformation parameters
def update_value(val, display_name):
    variables[slider_map[display_name]] = scales[display_name].get()

def apply_transformation(frame):
    #frame = frame[:frame.shape[0] - 15, 178:]
    h, w, _ = frame.shape
    H, W = 1080, 1920

    pad_y = (H - h)//2
    pad_x = (W - w)//2
    frame = cv2.copyMakeBorder(frame, pad_y, pad_y, pad_x, pad_x, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    x0, y0 = W // 2, H // 2  # Center of the image

    # Normalize coordinates from -1 to 1
    normalized_x = np.linspace(-W/w, W/w, W)
    normalized_y = np.linspace(-H/h, H/h, H)


    grid_x, grid_y = np.meshgrid(normalized_x, normalized_y)


    # Retrieve transformation parameters
    zx, zy = variables['zx'], variables['zy']  # Zoom X and Y
    px, py = variables['px'], variables['py']  # Perspective X and Y
    sx, sy = variables['sx'], variables['sy']  # Skew X and Y
    tx, ty = variables['tx'], variables['ty']  # Translation X and Y
    r = variables['r'] #rotation

    # Apply transformations
    # Perspective X - Left and Right sides adjust, middle stays fixed
    #x_prime = grid_x * (1 - zx)+((px)/2*grid_y+1)*grid_x + (sx * grid_y) + tx

    # Perspective Y - Top and Bottom sides adjust, middle stays fixed
    #y_prime = grid_y * (1 - zy)+((py)/2*grid_x+1)*grid_y + (sy * grid_x) + ty
  # Apply transformations

        # Zoom
    x_prime = grid_x*(1 - zx)
    y_prime = grid_y*(1 - zy)
    
    # Translation - Moves center to (tx,ty)
    x_prime += tx -0.085 #translates for the cropping.
    y_prime += ty
    
    # Skew
    x_prime += sx*grid_y
    y_prime += sy*grid_x
    
    # Perspective
    x_prime /= (0.5*px*grid_y + 1 ) #could remove the 0.5 so it can get further but need to limit values for px and py so they cant be +/- 1
    y_prime /= (0.5*py*grid_x +1 )
    
    # Rotation
    ((y_rot), (x_rot)) = ((0),(0)) 
    np.array([y_rot],[x_rot]) == np.array([[math.cos(r*math.pi/180), -math.sin(r*math.pi/180)],[ math.sin(r*math.pi/180), math.cos(r*math.pi/180)]])@np.array([y_prime],[x_prime])
    tup_rot = tuple.flatten([y_rot,x_rot])
    y_prime = tup_rot[0]
    x_prime = tup_rot[1]
    
    # Scale to image size
    map_x = (w*x_prime//2 + x0).astype(np.float32)
    map_y = (h*y_prime//2 + y0).astype(np.float32)



    # Apply remapping
    transformed = cv2.remap(frame, map_x, map_y, cv2.INTER_LINEAR)
    return transformed



def update_frame():
    
    img = Image.open(r"C:\Users\varun\OneDrive\Documents\Python Files\Warped StarTech Feed_screenshot_13.03.2025_no patient data.PNG")
    ret, frame = True, np.array(img)
    if ret:
        # Crop the frame (remove leftmost 178 pixels and bottommost 15 pixels. Gets rid of the patient info and just shows the image)
        frame = frame[:frame.shape[0] - 15, 178:]
        label.pack(expand=True, fill='both') #supposed to center the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame2 = apply_transformation(frame) #using Varun’s idea
        image_pil = Image.fromarray(frame2)
        image_tk = ImageTk.PhotoImage(image_pil)
        label.config(image=image_tk)
        label.image = image_tk
        # Move the image within the window using place()
        #label.place(x=44, y=7)  # Adjust x and y to move the image
    root.after(10, update_frame)


# Define sliders
slider_map = {"Zoom X": "zx", "Zoom Y": "zy", "Perspective X": "px", "Perspective Y": "py",
              "Skew X": "sx", "Skew Y": "sy", "Translation X": "tx", "Translation Y": "ty", "Rotation": "r"}

scales, labels = {}, {}
variables = {var_name: 0 for var_name in slider_map.values()}

# Create sliders
slider_window = tk.Toplevel(root)
slider_window.title("Adjust Transformations")
slider_window.geometry("400x600")

for display_name, var_name in slider_map.items():
    frame = tk.Frame(slider_window)
    frame.pack(pady=5, fill="x")
    labels[display_name] = tk.Label(frame, text=f"{display_name}: 0.00", width=15)
    labels[display_name].pack(side="left")
    
    if var_name == "r":
        from_, to, resolution = 0, 360, 1
    else:
        from_, to, resolution = -1, 1, 0.01
    
    scales[display_name] = tk.Scale(frame, from_=from_, to=to, resolution=resolution, orient="horizontal",
                                    command=lambda val, name=display_name: update_value(val, name))
    scales[display_name].pack(side="right", expand=True, fill="x")

# Bind the "Esc" key to exit the program
root.bind("<Escape>", lambda event: root.destroy())  # Press ESC to close
# Start video update
update_frame()
root.mainloop()
cap.release()


TypeError: Field elements must be 2- or 3-tuples, got '0'

In [1]:
#FRANK fixing the cropping issue
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import ctypes

ctypes.windll.shcore.SetProcessDpiAwareness(1)  # Enable DPI scaling fix

# Initialize Tkinter
root = tk.Tk()
root.title("Capture Card Feed with Transformations")
root.overrideredirect(True) #removes GUI elements/title bar
root.geometry("1920x1080") #size fits the projector screen
root.geometry('+2256+0') #moves window to start of projector screen
root.configure(bg="black")

# Open the Capture Card
cap = cv2.VideoCapture(1)  # Change index if necessary
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 812)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 650) #812x650 is the size of fluoro computer images
cap.set(cv2.CAP_PROP_FPS, 30)

# Create a label to display video
label = tk.Label(root, bg="black") #turned background of window black


# Transformation parameters
def update_value(val, display_name):
    variables[slider_map[display_name]] = scales[display_name].get()

def apply_transformation(frame):
    #frame = frame[:frame.shape[0] - 15, 178:]
    h, w, _ = frame.shape
    H, W = 1080, 1920

    pad_y = (H - h)//2
    pad_x = (W - w)//2
    frame = cv2.copyMakeBorder(frame, pad_y, pad_y, pad_x, pad_x, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    x0, y0 = W // 2, H // 2  # Center of the image

    # Normalize coordinates from -1 to 1
    normalized_x = np.linspace(-W/w, W/w, W)
    normalized_y = np.linspace(-H/h, H/h, H)


    grid_x, grid_y = np.meshgrid(normalized_x, normalized_y)


    # Retrieve transformation parameters
    zx, zy = variables['zx'], variables['zy']  # Zoom X and Y
    px, py = variables['px'], variables['py']  # Perspective X and Y
    sx, sy = variables['sx'], variables['sy']  # Skew X and Y
    tx, ty = variables['tx'], variables['ty']  # Translation X and Y

    # Apply transformations
    # Perspective X - Left and Right sides adjust, middle stays fixed
    #x_prime = grid_x * (1 - zx)+((px)/2*grid_y+1)*grid_x + (sx * grid_y) + tx

    # Perspective Y - Top and Bottom sides adjust, middle stays fixed
    #y_prime = grid_y * (1 - zy)+((py)/2*grid_x+1)*grid_y + (sy * grid_x) + ty
  # Apply transformations

        # Zoom
    x_prime = grid_x*(1 - zx)
    y_prime = grid_y*(1 - zy)
    
    # Translation - Moves center to (tx,ty)
    x_prime += tx -0.085 #translates for the cropping.
    y_prime += ty
    
    # Skew
    x_prime += sx*grid_y
    y_prime += sy*grid_x
    
    # Perspective
    x_prime /= (0.5*px*grid_y + 1 ) #could remove the 0.5 so it can get further but need to limit values for px and py so they cant be +/- 1
    y_prime /= (0.5*py*grid_x +1 )
    

    

    # Scale to image size
    map_x = (w*x_prime//2 + x0).astype(np.float32)
    map_y = (h*y_prime//2 + y0).astype(np.float32)



    # Apply remapping
    transformed = cv2.remap(frame, map_x, map_y, cv2.INTER_LINEAR)
    return transformed



def update_frame():
    ret, frame = cap.read()
    if ret:
        # Crop the frame (remove leftmost 178 pixels and bottommost 15 pixels. Gets rid of the patient info and just shows the image)
        frame = frame[:frame.shape[0] - 15, 178:]
        label.pack(expand=True, fill='both') #supposed to center the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame2 = apply_transformation(frame) #using Varun’s idea
        image_pil = Image.fromarray(frame2)
        image_tk = ImageTk.PhotoImage(image_pil)
        label.config(image=image_tk)
        label.image = image_tk
        # Move the image within the window using place()
        #label.place(x=44, y=7)  # Adjust x and y to move the image
    root.after(10, update_frame)


# Define sliders
slider_map = {"Zoom X": "zx", "Zoom Y": "zy", "Perspective X": "px", "Perspective Y": "py",
              "Skew X": "sx", "Skew Y": "sy", "Translation X": "tx", "Translation Y": "ty"}

scales, labels = {}, {}
variables = {var_name: 0 for var_name in slider_map.values()}

# Create sliders
slider_window = tk.Toplevel(root)
slider_window.title("Adjust Transformations")
slider_window.geometry("400x500")

for display_name, var_name in slider_map.items():
    frame = tk.Frame(slider_window)
    frame.pack(pady=5, fill="x")
    labels[display_name] = tk.Label(frame, text=f"{display_name}: 0.00", width=15)
    labels[display_name].pack(side="left")
    scales[display_name] = tk.Scale(frame, from_=-1, to=1, resolution=0.01, orient="horizontal",
                                    command=lambda val, name=display_name: update_value(val, name))
    scales[display_name].pack(side="right", expand=True, fill="x")

# Bind the "Esc" key to exit the program
root.bind("<Escape>", lambda event: root.destroy())  # Press ESC to close
# Start video update
update_frame()
root.mainloop()
cap.release()
